In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(
    temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

docs = loader.load_and_split(text_splitter=splitter)

# 1. Embedding
embeddings = OpenAIEmbeddings()
# 2. Cache
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings,
    cache_dir,
)
# Vector Store
vectorstore = FAISS.from_documents(docs, embeddings)

# Retriever document를 list로 반환한다.
retriever = vectorstore.as_retriever()

# 1. 질문을 하면 retriever에서 document list를 가져온다.
# 2. list내부의 모든 document들을 위한 prompt를 만든다.
# 3. prompt를 LLM에게 전달한다. (이 docment를 읽고 사용자의 질문에 답변하기에 적절한 정보가 있는지 확인해주세요.)
# 4. LLM으로 부터 전달받은 response들을 취합하여 하나의 document를 만든다.
# 5. 하나로 합쳐진 최종 document가 LLM을 위한 prompt로 전달한다.
# 6. 처음 질문에 대한 답변이 생성된다.

# retriever가 검색결과로 천개 이상의 document 를 반환한다면 stuff를 사용할 수 없다. stuff prompt에 모든 document를 넣을수 없기때문에


map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

map_doc_chain = map_doc_prompt | llm


def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    # case 1)
    # results = []
    # for document in documents:
    #     result = map_doc_chain.invoke(
    #         {"context": document.page_content, "question": question}
    #     ).content
    #     results.append(result)
    # results = "\n\n".join(results)
    # return results
    # case 2)
    return "\n\n".join(
        map_doc_chain.invoke(
            {"context": doc.page_content, "question": question}
        ).content
        for doc in documents
    )


map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer.
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            And Please translate it into Korean.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": map_chain,
        "question": RunnablePassthrough(),
    }
    | final_prompt
    | llm
)

chain.invoke("Where does Winston go to work")

AIMessage(content='윈스턴은 진실부에서 일합니다.')